In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [2]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (128, 128)

In [3]:
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)      # Resize to (128,128)
    image = tf.cast(image, tf.float32) / 255.0    # Normalize to [0,1]
    return image, label

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'beans',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/incomplete.70SWW9_0.1.0/beans-train.tfrecord*...:   0%|          | 0…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/incomplete.70SWW9_0.1.0/beans-validation.tfrecord*...:   0%|        …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/incomplete.70SWW9_0.1.0/beans-test.tfrecord*...:   0%|          | 0/…

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [5]:
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 classes (healthy, rust, spot)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_test)

Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 26s 848ms/step - accuracy: 0.3429 - loss: 1.5057 - val_accuracy: 0.4928 - val_loss: 1.0161
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 43s 919ms/step - accuracy: 0.5302 - loss: 1.0000 - val_accuracy: 0.6570 - val_loss: 0.8221
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 21s 818ms/step - accuracy: 0.6871 - loss: 0.7483 - val_accuracy: 0.6522 - val_loss: 0.7923
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 834ms/step - accuracy: 0.7605 - loss: 0.6244 - val_accuracy: 0.7150 - val_loss: 0.6673
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 23s 867ms/step - accuracy: 0.8191 - loss: 0.5197 - val_accuracy: 0.7101 - val_loss: 0.6620


In [9]:
loss, acc = model.evaluate(ds_test)
print(f"\nTest Accuracy: {acc * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 308ms/step - accuracy: 0.6997 - loss: 0.6931

Test Accuracy: 71.01%
